In [3]:
import numpy as np
import keras
import tensorflow as tf
from keras.models import Sequential, load_model, Model, Input
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPool2D, InputLayer, ZeroPadding2D, GlobalAvgPool2D, Reshape, Softmax
from keras.datasets import cifar10
from keras.applications.mobilenet import MobileNet
from keras.utils import np_utils
import matplotlib.pyplot as plt
from skimage.transform import resize
from keras import metrics


%matplotlib inline

In [ ]:
#  This tutorial attempts to achieve transfer learning on incompatible image sizes with MobileNet.
#  Why do this, when we could just load a different (compatible) data set, or use a different model?
#  Because!  Well, this is a tutorial, and it illustrates some practical issues implementing transfer learning.
#  We will resize a small batch of images to show how pre-trained weights affect fitting and prediction.
#  The unbiased and biased MobileNet objects will have the same layer architecture except for the weights.

#  Table of Contents:
#  
#  1.  Data Processing
#  2.  Create MobileNet Conv. Net. Models
#  3.  Prediction and Scoring
#


# 1.  Data Processing

## Load Data: (32,32,3) Images

In [ ]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

#  Check shapes.

In [ ]:
#  Convince yourself that the labels are consistent with the data.  
#  See https://www.cs.toronto.edu/~kriz/cifar.html
print(y_train[4999])   
plt.imshow(x_train[4999])

## Resize Train Data

In [ ]:
%%time

#  Pretrained weights only exist for certain shapes, which is why we get an error with smaller image sizes.
#  One option to deal with the fact that MobileNet does not like (32,32,3) shape is to resize the images.
#  NOTE:  On my machine, resizing the entire training data set would take, according to my precise calculations, a very long time.
#  I will only be resizing and training on 5,000 (about 50 minutes for me), creating a 2 gig file.  
#  You will have to do this on your own, I will not include the resized file on github (although it is possible to host large files).
#  Watch your memory usage.

#  Base case.
resized_train_data = []
resized_train_data = np.reshape(np.append(resized_train_data, resize(x_train[0],(128,128,3))),(128,128,3))

#  Change range to 50000 if you want to do the entire set.  NOT RECOMMENDED
for row in range(5000):
    if row > 0:
        resized = resize(x_train[row],(128,128,3))
        resized_train_data = np.reshape(np.append(resized_train_data, resized),((row+1),128,128,3))
          

In [ ]:
#  Check
plt.imshow(resized_train_data[4999])

## Resize Test Data

In [ ]:
#  Also resize test data
resized_test_data = []
resized_test_data = np.reshape(np.append(resized_test_data, resize(x_test[0],(128,128,3))),(128,128,3))

#  Change range to 10000 if you want to resize entire test data.  NR
for row in range(1000):
    if row > 0:
        resized = resize(x_test[row],(128,128,3))
        resized_test_data = np.reshape(np.append(resized_test_data, resized),((row+1),128,128,3))
          

In [ ]:
plt.imshow(x_test[17])

In [ ]:
plt.imshow(resized_test_data[17])

## Save/Load

In [ ]:
#  Since resizing takes time, you might want to save.  (Also perhaps for resized_test_data, but I omit these details.)
#np.save('resized_5k_images.npy',resized_train_data)

In [ ]:
#resized_images = np.load('resized_5k_images.npy')

#  Or, alternately if you saved as .npz
with np.load('/Users/Beebs/Desktop/resized_5k_images.npz') as data:
    train_images = data['arr_0']

In [ ]:
#  Check
train_images.shape

## Scale Data

In [ ]:
resized_train_images = train_images / 255

In [ ]:
#  Also for test data (which may have a different name if you are loading them.)
resized_test_images = resized_test_data / 255

## Encode Category Labels

In [ ]:
#  Encode labels.
y_train_encoded = np_utils.to_categorical(y_train)
y_test_encoded = np_utils.to_categorical(y_test)

In [ ]:
#  Grab relevant number of labels for resized batch.
fivek_labels = y_test_encoded[:5000]
fivek_labels.shape

# 2.  Create MobileNet Conv. Net. Models

## Create Unbiased Conv. NN

In [2]:
#  With the resized image shapes, we can use them directlly with MobileNet.  
#  Otherwise, it would throw an error that the input_shape is too small.
#  However, I am using a dataset of 5000, which does not seem to be enough to increase accuracy above chance.
#  This illustrates one of the uses of transfer learning:  when data sets are too small to properly train.

fresh = MobileNet(dropout=0,input_shape=(128,128,3),include_top =True, weights=None,classes=10)
fresh.summary()
fresh.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 128, 128, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 130, 130, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 64, 64, 32)        864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 64, 64, 32)        128       
_________________________________________________________________
conv1_relu (Activation)      (None, 64, 64, 32)        0         
_________________________________________________________________
conv_pad_1 (ZeroPadding2D)   (None, 66, 66, 32)        0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 64, 64, 32)        288       
__________

## Fit

In [ ]:
%%time

#  Fitting takes a long time.  
#  If you want to interrupt the cell, click left outside the text field (or hit Esc) and type 'ii'.
history_fresh = fresh.fit(resized_train_images,fivek_labels,epochs = 10,batch_size=1)

## Plot

In [ ]:
plt.plot(history_fresh.history['loss'])
#plt.plot(history_fresh.history['acc'])

## Create Biased Conv. NN

In [ ]:
#  Since we are doing a more complex model, we use the functional API Model() class from keras.
#  We manually describe the inputs and outputs.
#  For this MobileNet object, we will need to chop off the classification layers when 
#  using pre-trained weights.    
    
#  Create tensor object.
inputs = Input(shape = (128,128,3)) 

#  
trained_model = MobileNet(dropout=0,input_shape = (128,128,3),include_top = False, weights='imagenet', input_tensor = inputs)

#  Freeze: keep pre-trained weights as they are.
#  Check number of trainable parameters in summary after freezing layers.
for layer in trained_model.layers:
    layer.trainable = False

#  We can just plug the biased model into a layer.
#x = trained_model(inputs)

#  Then, if we want we can copy as close as possible the 
#  structure of the layers removed by include_top=False.

x = GlobalAvgPool2D(data_format='channels_last')(trained_model.output)
x = Reshape((1,1,-1))(x)
x = Dropout(rate=0.001)(x)
x = Conv2D(filters=10,kernel_size=(1,1))(x)
x = Activation(activation = 'softmax')(x)
predictions = Reshape((-1,))(x)


transfer_model = Model(inputs = inputs,outputs = predictions)
transfer_model.compile(optimizer = 'adam',loss = 'categorical_crossentropy',metrics=['accuracy'])
transfer_model.summary()

In [ ]:
%%time

#  Notice fitting the pre-trained model is much quicker, each epoch takes much less time.
#  It also begins increasing accuracy with the small sample set I am using (5000).
history_transfer = transfer_model.fit(resized_train_images,fivek_labels,epochs=10,batch_size=1)

## Compare Performance

In [ ]:
plt.plot(history_transfer.history['loss'])

In [ ]:
plt.plot(history_transfer.history['acc'])

In [ ]:
#  Compare with plots from unbiased network.
plt.plot(history_fresh.history['loss'])

In [ ]:
plt.plot(history_fresh.history['acc'])

# 3.  Prediction and Scoring

In [ ]:
#  Supress scientific notation for easier comparison.
np.set_printoptions(suppress=True)

#  Predict a class and look at an example to compare between biased and unbiased.
#  What do you expect the comparison to show?
unbiased_prediction = fresh.predict(resized_test_images)

In [ ]:
unbiased_prediction[72]

In [ ]:
#  Note: predict gives us probabilities like predict_proba for other models.  Check that they sum to one.
sum(unbiased_prediction[72])

In [ ]:
biased_prediction = transfer_model.predict(resized_test_images)
biased_prediction[72]

In [ ]:
#  True label:
y_test_encoded[72]

In [ ]:
#  Unbiased
#  Brier score, lower is better: smaller distance between prediction and true label.  
#  Try looking at prediction scores before and after training, and after different amounts of training.
unbiased_diff = y_test_encoded[:1000] - unbiased_prediction
score_u = np.sum((1/1000)*(np.power(unbiased_diff,2)),axis=1)

#  Overall score for 1000 test examples.
sum(score_u)

In [ ]:
#  Biased
biased_diff = y_test_encoded[:1000] - biased_prediction  
score_b = np.sum((1/1000)*(np.power(biased_diff,2)),axis=1)

sum(score_b)

In [ ]:
metrics.mean_squared_error(y_te)